In [1]:
import numpy as np
import matplotlib.pyplot as plt

# # input_question = "I'm going to classify the following chinese medicine spectral data."
# input_question = "I'm going to anomaly detect the chinese medicine spectral data."
# data = np.load(r'data\CN_medicine\cnml.npy')

# input_question = "I'm going to classify the following Citri Reticulatae Pericarpium spectral data."
# # input_question = "I'm going to anomaly detect the Citri Reticulatae Pericarpium spectral data."
# data = np.load(r'data\Chenpi\chenpi.npy')

input_question = "I'm going to classify the following milk spectral data."
# input_question = "I'm going to anomaly detect the following milk spectral data."
data = np.load(r'data\milk\milk_data.npy')

# input_question = "I'm going to predict the waste water spectral data quality of COD."
# data = np.load(r'data\H2O\H2Odata_js.npy')
# Y = np.load(r'data\H2O\H2Olabel_js.npy')


# input_question = "I'm going to predict the fat content in meat sample."
# X = np.load(r'data\tecator\tecator_data.npy')
# Y = np.load(r'data\tecator\tecator_label.npy')
# data=X.reshape(1, X.shape[0], -1)
# Y = Y.reshape(1, 215, 1)


# input_question = "I'm going to predict the protein content in corn samples."
# data = np.load(r'data\corn\corn_data.npy')
# Y = np.load(r'data\corn\protein_label.npy')
# data=data.reshape(1, data.shape[0], -1)
# Y = Y.reshape(1, Y.shape[1], 1)
# regression_label = Y

print(data.shape)


API = "sk-25924755df8b4a5e900e4eec74caf179"
base_url = "https://dashscope.aliyuncs.com/compatible-mode/v1"

## GPT ds API
model_API =  "sk-GodYd4JhmouwdGxgoRJAmlFKvbyImUubriA0ZVgW7faNhOnl"
model_url = "https://jeniya.top/v1"

(9, 40, 601)


In [2]:
import Entity_extraction
extracted = Entity_extraction.extract_entities_and_task(input_question, API, base_url, model_name="qwen-plus")

research_object = extracted['research_object']
task_type       = extracted['task_type']

print("Research object:", research_object)
print("Task type:", task_type)

Research object: milk
Task type: classification


In [3]:
import Retrieval

json_path = "./structured_papers1.json"

papers_list = Retrieval.load_papers_from_json(json_path)

bm25_index, tokenized_names = Retrieval.build_bm25_index(papers_list)
top_k = 3

matched_info = Retrieval.search_papers_with_bm25(
    papers=papers_list,
    bm25=bm25_index,
    tokenized_paper_names=tokenized_names,
    query=research_object,
    top_k=top_k
)

print(matched_info)

Paper_Name: Milk Classification of NIR spectra using Principal Component Analysis in Python
Relevant Scores: 5.948412985878816
Paper_Name: Identification of the animal origin of milk via Laser-Induced Breakdown Spectroscopy
Relevant Scores: 5.210590843908991
Paper_Name: Novel prediction models for hyperketonemia using bovine milk Fourier-transform infrared spectroscopy
Relevant Scores: 4.122371872440508
[{'paper_name': 'Milk Classification of NIR spectra using Principal Component Analysis in Python', 'preprocessing_method': 'SG', 'feature_extracting_method': 'PCA'}, {'paper_name': 'Identification of the animal origin of milk via Laser-Induced Breakdown Spectroscopy', 'preprocessing_method': 'Feature-wise scaling', 'feature_extracting_method': 'Entire LIBS spectra'}, {'paper_name': 'Novel prediction models for hyperketonemia using bovine milk Fourier-transform infrared spectroscopy', 'preprocessing_method': 'scaling and centering data (μ=0, σ^2=1), Synthetic Minority Oversample Techniqu

In [4]:
import Agent

Default_mode = True

if Default_mode:
    agent = Agent.SpectralAgent(
        api_key=API,
        base_url=base_url,
        model_name="qwen-plus",
        regression_label=Y if task_type == 'regression' else None,
    )

    methods_map = agent.decide_methods_per_paper(matched_info)
    print(methods_map)

    paper_order = [m['paper_name'] for m in matched_info] 

    # 1) Standard usage: majority voting among top K; if tied or zero votes, fall back to the first "complete" paper among top K
    selected = agent.select_methods(
        methods_map,
        k=3,
        paper_order=paper_order,
        require_both=True,   # Require both "preprocessing + features"
    )
    print("Selected (majority + fallback):", selected)


else:
# Free selection (by index)
    selected_by_idx = agent.select_methods(
        methods_map,
        k=3,
        paper_order=paper_order,
        prefer_paper=2,       # Pass 2 to select the 2nd paper; pass 1 or 0 to indicate 1-based/0-based indexing for the 1st paper
        prefer_strict=True,
        require_both=True,
    )
    print("Selected (prefer by index):", selected_by_idx)

# Process data and extract features using selected methods
processed_data, features = agent.process_with_methods(data, selected)
print(processed_data.shape)
for fn, arr in features.items():
    print(f"Feature '{fn}' shape:", arr.shape)

{'Milk Classification of NIR spectra using Principal Component Analysis in Python': {'preprocessing': ['savitzky_golay_smoothing'], 'features': ['pca_feature_extraction']}, 'Identification of the animal origin of milk via Laser-Induced Breakdown Spectroscopy': {'preprocessing': [], 'features': []}, 'Novel prediction models for hyperketonemia using bovine milk Fourier-transform infrared spectroscopy': {'preprocessing': [], 'features': []}}
Selected (majority + fallback): {'preprocessing': ['savitzky_golay_smoothing'], 'features': ['pca_feature_extraction']}
(9, 40, 601)
Feature 'pca_feature_extraction' shape: (9, 40, 5)


In [5]:
from dataset import CLS_Dataset, REG_Dataset, ANO_Dataset
from dataset_config import resolve_dataset_config

split_ratio = [0.6, 0.2, 0.2]

cfg = resolve_dataset_config(research_object)
labels = cfg["labels"]
random_seed = cfg["seed"]

input = list(features.values())[0]
input_len = input.shape[0] * input.shape[1]
if task_type == 'regression':
    input_len = data.shape[1]
    total_samples = 100 if input_len > 100 else input_len
else:
    total_samples = input_len

if task_type == 'classification':
    dataset = CLS_Dataset(
        feature=input,
        labels=labels,
        total_samples=total_samples,
        split_ratio=split_ratio,
        random_seed=random_seed
    )
    dataset.summary()
    train_data = dataset.train_data
    val_data   = dataset.val_data
    test_data  = dataset.test_data
    true_labels_val  = dataset.true_labels_val
    true_labels_test = dataset.true_labels_test

elif task_type == 'regression':
    dataset = REG_Dataset(
        X=input,
        Y=Y,
        total_samples=total_samples,
        split_ratio=split_ratio,
        random_seed=random_seed,
    )
    dataset.summary()
    train_data = dataset.train_data
    val_data   = dataset.val_data
    test_data  = dataset.test_data
    y_val_true  = dataset.y_val_true
    y_test_true = dataset.y_test_true

elif task_type == 'anomaly detection':
    ds = ANO_Dataset(
        X=input,
        labels=labels,
        total_samples=total_samples,
        split_ratio=split_ratio,
        normal_class=0,
        random_seed=random_seed,
    )
    ds.summary()
    train_data = ds.train_data
    val_data   = ds.val_data
    test_data  = ds.test_data
    y_val      = ds.y_val
    y_test     = ds.y_test


Training samples: 216
Validation samples: 72, labels: ['5', '1', '6', '5', '2', '2', '4', '7', '4', '9', '8', '8', '3', '9', '1', '2', '2', '9', '4', '7', '7', '9', '9', '6', '6', '5', '3', '2', '6', '9', '7', '3', '4', '8', '8', '4', '8', '5', '2', '8', '1', '9', '6', '8', '5', '6', '1', '7', '1', '7', '1', '3', '7', '3', '4', '2', '5', '4', '6', '3', '5', '5', '7', '9', '2', '6', '3', '3', '1', '8', '1', '4']
Test samples: 72, labels: ['2', '2', '8', '8', '1', '6', '4', '6', '1', '9', '6', '5', '9', '7', '2', '5', '8', '2', '9', '7', '1', '9', '6', '1', '6', '3', '3', '6', '1', '3', '9', '4', '3', '5', '1', '5', '7', '3', '1', '2', '3', '8', '3', '2', '4', '9', '2', '5', '4', '5', '8', '7', '3', '7', '4', '9', '8', '7', '2', '8', '9', '6', '6', '4', '5', '4', '1', '7', '5', '8', '7', '4']


In [6]:
## Single turn generation
from Generate_single import SpectrumCLS, SpectrumReg, SpectrumAno

if task_type == "classification":
    agent_cls = SpectrumCLS(
        train_data=train_data,
        test_data=test_data,
        true_labels_test=true_labels_test,
        api_key=model_API,
        base_url=model_url,
        model="gpt-5-chat-latest",
        #model = "qwen-turbo-latest",
    )
    agent_cls.run()

elif task_type == "regression":
    agent_reg = SpectrumReg(
        dataset= dataset,
        api_key=model_API,
        base_url=model_url,
        model="gpt-5-chat-latest",
        #model = "qwen-plus-latest",
    )
    agent_reg.run()

elif task_type == 'anomaly detection':
    agent_ano = SpectrumAno(
        dataset = ds,
        api_key=model_API,
        base_url=model_url,
        model="gpt-5-chat-latest",
        #model = "qwen-plus-latest",
    )
    agent_ano.run()

=== Initial test set evaluation ===
Prompt tokens: 16605, Completion tokens: 148
raw_model_output: [2,2,8,8,1,6,3,6,1,9,6,5,9,7,2,5,8,2,9,7,1,9,6,1,6,3,3,6,1,3,9,4,3,5,1,5,7,3,1,2,3,8,3,2,4,9,2,5,4,5,8,7,3,7,4,9,8,7,2,8,9,7,6,6,4,5,4,1,7,5,8,7,4]
parsed_preds: ['2', '2', '8', '8', '1', '6', '3', '6', '1', '9', '6', '5', '9', '7', '2', '5', '8', '2', '9', '7', '1', '9', '6', '1', '6', '3', '3', '6', '1', '3', '9', '4', '3', '5', '1', '5', '7', '3', '1', '2', '3', '8', '3', '2', '4', '9', '2', '5', '4', '5', '8', '7', '3', '7', '4', '9', '8', '7', '2', '8', '9', '7', '6', '6', '4', '5', '4', '1', '7', '5', '8', '7']
true_labels_test: ['2', '2', '8', '8', '1', '6', '4', '6', '1', '9', '6', '5', '9', '7', '2', '5', '8', '2', '9', '7', '1', '9', '6', '1', '6', '3', '3', '6', '1', '3', '9', '4', '3', '5', '1', '5', '7', '3', '1', '2', '3', '8', '3', '2', '4', '9', '2', '5', '4', '5', '8', '7', '3', '7', '4', '9', '8', '7', '2', '8', '9', '6', '6', '4', '5', '4', '1', '7', '5', '8', '7', '4']

In [7]:
## Other models

from other_models import ClassificationModelPipeline,RegressionModelPipeline,AnomalyModelPipeline

if task_type =='classification':
    cls_pipe = ClassificationModelPipeline(train_data, test_data, val_data=val_data, early_stop_patience=5, cv_folds=5)
    cls_results = cls_pipe.train_and_evaluate(nn_epochs=50, batch_size=32)

elif task_type =='regression': 
    reg_pipe = RegressionModelPipeline(train_data, test_data, val_data=val_data, early_stop_patience=5, cv_folds=5)
    reg_results = reg_pipe.train_and_evaluate(nn_epochs=50, batch_size=32)
    
elif task_type == 'anomaly detection':
    ano_pipe = AnomalyModelPipeline(train_data, test_data, val_data=val_data, early_stop_patience=5, cv_folds=5)
    ano_results = ano_pipe.train_and_evaluate(epochs_ae=50, batch_size=32)

d:\Conda\envs\pytorch\Lib\site-packages\torch\utils\_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


[ML][SVM] best_params={'svc__C': 100, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}, CV_best_acc=1.0000, Test_acc=1.0000
[ML][KNN] best_params={'knn__n_neighbors': 7, 'knn__p': 1, 'knn__weights': 'distance'}, CV_best_acc=0.9400, Test_acc=0.9306
[ML][RandomForest] best_params={'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}, CV_best_acc=0.9537, Test_acc=0.9722
[NN-CLS] Early stopped at epoch 12, best val_acc=0.9583
[NN-CLS] Early stopped at epoch 23, best val_acc=1.0000
SVM: Accuracy = 100.00%
KNN: Accuracy = 93.06%
RandomForest: Accuracy = 97.22%
CNN1D: Accuracy = 91.67%
Transformer: Accuracy = 97.22%
